In [1]:
import altair as alt
import pandas as pd

# Importing the plugin
import persist_ext as PR

In [2]:
df = pd.read_csv("avalanches_data.csv") # Load avalanche data
df.head() # See first few rows

,;Region,Year,;Trigger,;Weak Layer,Depth_inches,;Aspect,Day,Elevation_feet,Vertical_inches,Month
0,Salt Lake,2012,Snowboarder,New Snow/Old Snow Interface,14.0,North,9,10400.0,360.0,11
1,Salt Lake,2012,Skier,New Snow/Old Snow Interface,30.0,North,11,9700.0,1200.0,11
2,Salt Lake,2012,Skier,Facets,36.0,North,11,10200.0,5400.0,11
3,Salt Lake,2012,Skier,New Snow,"18.0""",Southeast,11,10200.0,6000.0,11
4,Salt Lake,2012,Skier,Facets,42.0,North,11,10400.0,9600.0,11


In [3]:
# Create interactive data table using Persist
PR.PersistTable(df)

PersistWidget(data_values=[{'__id_column': '1', ';Region': 'Salt Lake', 'Year': 2012, ';Trigger': 'Snowboarder…

In [4]:
persist_df_1.head()

,Region,Year,Trigger,Weak Layer,Depth_inches,Aspect,Day,Elevation_feet,Vertical_inches,Month
0,Salt Lake,2012,Snowboarder,New Snow/Old Snow Interface,14.0,North,9,10400.0,360.0,11
1,Salt Lake,2012,Skier,New Snow/Old Snow Interface,30.0,North,11,9700.0,1200.0,11
2,Salt Lake,2012,Skier,Facets,36.0,North,11,10200.0,5400.0,11
3,Salt Lake,2012,Skier,New Snow,18.0,Southeast,11,10200.0,6000.0,11
4,Salt Lake,2012,Skier,Facets,42.0,North,11,10400.0,9600.0,11


In [5]:
# Use the dataframe `persist_df_1` to create scatterplot using `plot` module in Persist

PR.plot.scatterplot(
    persist_df_1,
    x="Elevation_feet:Q",
    y="Depth_inches:Q",
    opacity=alt.value(0.5)
)

PersistWidget(data_values=[{'__id_column': '1', 'Region': 'Salt Lake', 'Year': 2012, 'Trigger': 'Snowboarder',…

In [8]:
no_outliers_df.head()

,Region,Year,Trigger,Weak Layer,Depth_inches,Aspect,Day,Elevation_feet,Vertical_inches,Month
0,Salt Lake,2012,Snowboarder,New Snow/Old Snow Interface,14,North,9,10400.0,360.0,11
1,Salt Lake,2012,Skier,New Snow/Old Snow Interface,30,North,11,9700.0,1200.0,11
2,Salt Lake,2012,Skier,Facets,36,North,11,10200.0,5400.0,11
3,Salt Lake,2012,Skier,New Snow,18,Southeast,11,10200.0,6000.0,11
4,Salt Lake,2012,Skier,Facets,42,North,11,10400.0,9600.0,11


In [9]:
no_outliers_df["Elevation_feet"].describe()

count         2325.0
mean     9510.666667
std      1110.390503
min           5100.0
25%           9000.0
50%           9700.0
75%          10200.0
max          12200.0
Name: Elevation_feet, dtype: Float64

In [10]:
# Rectangular brush object
brush = alt.selection_interval(name="brush", encodings=["x"])

# Create barchart for records aggregated across months using Vega-Altair
barchart = (
    alt.Chart()
    .mark_bar()
    .encode(
        x=alt.X("Month:N").sort([10]),
        y="count()",
        opacity=alt.condition(brush, alt.value(1), alt.value(0.3))
    ).add_params(brush)
    .properties(width=300)
)

# Create scatterplot for Elevation vs Vertical height of avalanche using Vega-Altair
scatterplot = (
    alt.Chart()
    .mark_point()
    .encode(
        x="Elevation_feet:Q",
        y="Vertical_inches:Q",
        opacity=alt.condition(brush, alt.value(0.4), alt.value(0.1))
    )
)

# Compose the barchart and scatterplot into a horizontally concatenated composite chart
# Set the chart to use the `no_outliers_df` dataframe generated previously
composite_chart = (
    alt.hconcat(barchart, scatterplot)
        .properties(data=no_outliers_df)
)

PR.PersistChart(composite_chart)

PersistWidget(data_values=[{'__id_column': '1', 'Region': 'Salt Lake', 'Year': 2012, 'Trigger': 'Snowboarder',…

In [11]:
persist_df_3.head()

,Region,Year,Trigger,Weak Layer,Depth_inches,Aspect,Day,Elevation_feet,Vertical_inches,Month
0,Salt Lake,2012,Snowboarder,New Snow/Old Snow Interface,14,North,9,10400,360,11
1,Salt Lake,2012,Skier,New Snow/Old Snow Interface,30,North,11,9700,1200,11
2,Salt Lake,2012,Skier,Facets,36,North,11,10200,5400,11
3,Salt Lake,2012,Skier,New Snow,18,Southeast,11,10200,6000,11
4,Salt Lake,2012,Skier,Facets,42,North,11,10400,9600,11


In [12]:
persist_df_3["Avalanche Season"].value_counts()

Avalanche Season
Middle           1581
Start             522
End               222
No Assignment       0
Name: count, dtype: int64

In [15]:
# Create `legend` selection object
selection = alt.selection_point(name="selection", fields=["Avalanche Season"], bind="legend")

# Base for all charts
base = alt.Chart()

# barchart for the new `Avalanche Season` category
av_season = base.mark_bar().encode(
        x=alt.X("Avalanche Season").sort(["Start", "Middle", "End"]),
        y="count()",
        color=alt.Color("Avalanche Season").sort(["Start", "Middle", "End"]),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
    ).add_params(selection).properties(width=150)


# Base for further charts
h_base = base.mark_bar(opacity=0.3).encode(y="count()").properties(width=250)

# Background for trigger barchart
trigger_bg = h_base.encode(x="Trigger:N")

# Foreground for trigger barchart
## reset opacity to default for foreground
trigger_fg = trigger_bg.mark_bar().transform_filter(selection)

# Layer background and foreground
trigger = trigger_bg + trigger_fg

# Background for weak layer barchart
weak_layer_bg = h_base.encode(x="Weak Layer:N")

# Foreground for weak layer barchart
## reset opacity to default for foreground
weak_layer_fg = weak_layer_bg.mark_bar().transform_filter(selection)

# Layer background and foreground
weak_layer = weak_layer_bg + weak_layer_fg

# Horizontally concatenate all three barcharts and and assign `persist_df_3` as data
chart = alt.hconcat(
    trigger,
    weak_layer,
    av_season
).properties(data=persist_df_3).resolve_scale(color="independent")

chart

alt.HConcatChart(...)